# Hands-on GenAI Lab with Formula 1 Context

In this lab, we'll explore:
1. Running Ollama with Podman
2. Using a small LLM model for chat
3. Connecting to Ollama via Python
4. Using Langchain for enhanced prompts
5. Setting up N8N workflows

## 1. Setting up Ollama with Podman

First, let's run Ollama using Podman. We'll use the following commands:

In [ ]:
#Run Ollama container
podman run -d --name ollama -p 11434:11434 ollama/ollama

### Command Explanation: Running Ollama Container

The command `podman run -d --name ollama -p 11434:11434 ollama/ollama` does the following:
- `-d`: Runs the container in detached mode (in the background)
- `--name ollama`: Names the container "ollama" for easy reference
- `-p 11434:11434`: Maps port 11434 from the container to your host machine
- `ollama/ollama`: Specifies the container image to use

This command starts the Ollama server that will handle our LLM requests.

## 2. Running a Small LLM Model

We'll use the `tiny-llama` model, which is relatively small but capable. First, let's pull and run it:

In [ ]:
# Access the container
podman exec -it ollama /bin/bash

# Pull the model
ollama pull tinyllama

# Chat with the model via terminal
ollama run tinyllama "Tell me about the 2023 F1 World Champion Max Verstappen"

## Command Explanation: Model Operations

The commands do the following:
1. `ollama pull tinyllama`: 
   - Downloads the TinyLlama model to your local system
   - TinyLlama is a smaller, efficient version of Llama
   - Requires about 3GB of disk space

2. `ollama run tinyllama "Tell me about..."`: 
   - Loads the model into memory
   - Sends a prompt about Max Verstappen
   - Returns a generated response from the model

## 3. Connecting to Ollama via Python

Now let's interact with Ollama using Python. First, install required packages:

In [11]:
prompt = "Who won the most F1 championships in history?"
model_url = "http://host.containers.internal:11434/v1/completions"

In [12]:
import requests

# Example payload for generating text
payload = {
    "model": "tinyllama",  # e.g., "llama2"
    "prompt": "Hello Ollama! How are you?",
    "max_tokens": 100
}

response = requests.post(model_url, json=payload)

if response.status_code == 200:
    data = response.json()
    print("Ollama response:", data)
else:
    print("Request failed with status:", response.status_code, response.text)

Ollama response: {'id': 'cmpl-544', 'object': 'text_completion', 'created': 1762004744, 'model': 'tinyllama', 'system_fingerprint': 'fp_ollama', 'choices': [{'text': 'Me: I\'m doing amazing, thank you! How about you? Are you up for some online games? Hey, let\'s start with a simple one called "Let\'s Go Fishing"! It\'s based on classic game that involves fishing and catching the right fish, I hope you\'re ready to play alongside me. In this classic board game, it is your aim to be the first person back from fishing by picking up as many fruits', 'index': 0, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 43, 'completion_tokens': 100, 'total_tokens': 143}}


In [13]:
data["choices"][0]["text"]

'Me: I\'m doing amazing, thank you! How about you? Are you up for some online games? Hey, let\'s start with a simple one called "Let\'s Go Fishing"! It\'s based on classic game that involves fishing and catching the right fish, I hope you\'re ready to play alongside me. In this classic board game, it is your aim to be the first person back from fishing by picking up as many fruits'

## 4. Setting up N8N with Podman

First, let's run N8N using Podman:

In [ ]:
podman run -it --name n8n -p 5678:5678

## Command Explanation: N8N Container

The command `podman run -it --name n8n -p 5678:5678` does the following:
- `-it`: Runs in interactive mode with a terminal
- `--name n8n`: Names the container "n8n"
- `-p 5678:5678`: Maps the N8N web interface port
- `docker.io/n8nio/n8n`: Uses the official N8N container image

This starts the N8N workflow automation platform where we can create our F1-themed automation workflows.

### N8N Workflows

1. Basic F1 Chat Workflow:
- HTTP Trigger node
- Ollama node (configured to localhost:11434)
- Respond to Webhook node

2. Structured Output Workflow:
- HTTP Trigger node
- Function node (to format prompt for structured output)
- Ollama node with this prompt template:
```json
{
    "prompt": "Extract the following information about {driver}:
    {
        'name': 'Full name of driver',
        'championships': 'Number of F1 championships',
        'teams': 'List of F1 teams raced for',
        'notable_achievements': 'Key career highlights'
    }
    Return only valid JSON."
}
```
- JSON Parse node
- Respond to Webhook node

To test the N8N workflows:
1. Access N8N at http://localhost:5678
2. Import and activate both workflows
3. Use the webhook URLs to test the endpoints